In [3]:
import torch
print(torch.version.cuda)


12.1


In [5]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [7]:
import torch
print(torch.__version__)


2.5.0+cu121


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, f1_score
import os
import numpy as np
import matplotlib.pyplot as plt

# Define the DFN-PSAN Model
def dfn_psan_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Multi-Level Convolution Blocks (Example with two levels)
    # First Convolution Block
    x1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
    x1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
    x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
    
    # Second Convolution Block (Higher resolution, different scale)
    x2 = layers.Conv2D(32, (5, 5), activation='relu', padding='same')(inputs)
    x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)
    x2 = layers.Conv2D(64, (5, 5), activation='relu', padding='same')(x2)
    x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)

    # Reshape the feature maps before applying Attention
    # Flatten the spatial dimensions for attention (batch, height*width, channels)
    x1_reshaped = layers.Reshape((-1, 64))(x1)  # Flatten spatial dimensions of x1
    x2_reshaped = layers.Reshape((-1, 64))(x2)  # Flatten spatial dimensions of x2

    # Apply Attention Mechanism
    attention_block_1 = layers.Attention()([x1_reshaped, x1_reshaped])  # Attention over x1 features
    attention_block_2 = layers.Attention()([x2_reshaped, x2_reshaped])  # Attention over x2 features
    
    # Reshape back after attention to the original spatial dimensions
    x1_attended = layers.Reshape((x1.shape[1], x1.shape[2], 64))(attention_block_1)
    x2_attended = layers.Reshape((x2.shape[1], x2.shape[2], 64))(attention_block_2)

    # Feature Fusion Layer: Combine features from both branches
    fusion = layers.Concatenate()([x1_attended, x2_attended])

    # Global Average Pooling for dimensionality reduction
    x = layers.GlobalAveragePooling2D()(fusion)

    # Fully Connected Classifier Layer
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    
    # Output Layer (For multi-class classification)
    outputs = layers.Dense(num_classes, activation='softmax')(x)  # Softmax for multi-class
    
    # Create the Model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Parameters
input_shape = (224, 224, 3)  # Example image size (224x224 with 3 color channels)
num_classes = 38  # Number of plant disease categories (assuming 38 classes)

# Create the model
model = dfn_psan_model(input_shape, num_classes)
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up ImageDataGenerators for training and validation data
train_dir = 'C:/Users/admin/Desktop/kaustubh/project/plant_diseases/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'  # Path to the training directory (should contain subfolders for each class)
val_dir = 'C:/Users/admin/Desktop/kaustubh/project/plant_diseases/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'      # Path to the validation directory (should contain subfolders for each class)

# Image data generators for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory for train and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Save the trained model
model.save('plant_disease_model.h5')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

# Get predictions on the validation data
val_predictions = model.predict(val_generator, verbose=1)
val_true = val_generator.classes

# Convert predictions to class labels (highest probability)
val_pred_labels = np.argmax(val_predictions, axis=1)

# Calculate Precision, Recall, F1-score, and F2-score
precision = precision_score(val_true, val_pred_labels, average='weighted')
recall = recall_score(val_true, val_pred_labels, average='weighted')
f1 = f1_score(val_true, val_pred_labels, average='weighted')
f2 = f1_score(val_true, val_pred_labels, average='weighted', beta=2)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'F2-Score: {f2}')

# Plot training history (optional)
plt.figure(figsize=(12, 6))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 224, 224,  │        896 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 224, 224,  │      2,432 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 112, 112,  │          0 │ conv2d_8[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_10    │ (None, 112, 112,  │          0 │ conv2d_10[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 112, 112,  │     18,496 │ max_pooling2d_8[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 112, 112,  │     51,264 │ max_pooling2d_10… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 56, 56,    │          0 │ conv2d_9[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 56, 56,    │          0 │ conv2d_11[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 3136, 64)  │          0 │ max_pooling2d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 3136, 64)  │          0 │ max_pooling2d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_4         │ (None, 3136, 64)  │          0 │ reshape[0][0],    │
│ (Attention)         │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_5         │ (None, 3136, 64)  │          0 │ reshape_1[0][0],  │
│ (Attention)         │                   │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 56, 56,    │          0 │ attention_4[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 56, 56,    │          0 │ attention_5[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 56, 56,    │          0 │ reshape_2[0][0],  │
│ (Concatenate)       │ 128)              │            │ reshape_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ concatenate_2[0]… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 100,326 (391.90 KB)

 Trainable params: 100,326 (391.90 KB)

 Non-trainable params: 0 (0.00 B)

Found 70243 images belonging to 38 classes.
Found 17557 images belonging to 38 classes.
Epoch 1/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 3218s 1s/step - accuracy: 0.1922 - loss: 2.8634 - val_accuracy: 0.4705 - val_loss: 1.7203
Epoch 2/10
   1/2195 ━━━━━━━━━━━━━━━━━━━━ 36:14 991ms/step - accuracy: 0.3125 - loss: 2.0209

C:\ProgramData\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


2195/2195 ━━━━━━━━━━━━━━━━━━━━ 1s 96us/step - accuracy: 0.3125 - loss: 2.0209 - val_accuracy: 0.6190 - val_loss: 1.6717
Epoch 3/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 2549s 1s/step - accuracy: 0.5422 - loss: 1.4819 - val_accuracy: 0.5778 - val_loss: 1.3416
Epoch 4/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 1s 93us/step - accuracy: 0.9062 - loss: 0.5937 - val_accuracy: 0.6190 - val_loss: 1.6080
Epoch 5/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 2573s 1s/step - accuracy: 0.7020 - loss: 0.9480 - val_accuracy: 0.5647 - val_loss: 1.5692
Epoch 6/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 1s 86us/step - accuracy: 0.8125 - loss: 0.6798 - val_accuracy: 0.4762 - val_loss: 1.9012
Epoch 7/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 2545s 1s/step - accuracy: 0.7748 - loss: 0.7141 - val_accuracy: 0.7552 - val_loss: 0.7817
Epoch 8/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 1s 86us/step - accuracy: 0.9375 - loss: 0.3525 - val_accuracy: 0.6667 - val_loss: 1.0734
Epoch 9/10
2195/2195 ━━━━━━━━━━━━━━━━━━━━ 2548s 1s/step - accuracy: 0.8151 - loss: 0.5798 -

549/549 ━━━━━━━━━━━━━━━━━━━━ 144s 263ms/step - accuracy: 0.8062 - loss: 0.6183
Validation Loss: 0.6047375202178955
Validation Accuracy: 0.8086802959442139
549/549 ━━━━━━━━━━━━━━━━━━━━ 145s 264ms/step


TypeError: got an unexpected keyword argument 'beta'